In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import convolutional,pooling,Dense,Flatten,BatchNormalization,Reshape,Dropout
from keras.utils import to_categorical
from keras.optimizers import Adam
import numpy as np
import skimage
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Getting the data and resizing the images

In [0]:
image_path =  '/content/drive/My Drive/PGAIML/CNN/Project1/plant_seed_classification/train/'

In [0]:
image_size = 128

In [0]:
classes = {'Black-grass':0,'Charlock':1,'Cleavers':2,'Common Chickweed':3,'Common wheat':4,'Fat Hen':5,'Loose Silky-bent':6,'Maize':7,
           'Scentless Mayweed':8,'Shepherds Purse':9,'Small-flowered Cranesbill':10,'Sugar beet':11}

In [0]:
img_count = 0
tot_img_count = 0
class_count = 0
for every_class in classes.keys():
  every_collection = skimage.io.imread_collection(image_path+every_class+'/*.png',conserve_memory=True)
  class_count+=1 
  for every_image in every_collection:
    img_count+=1
    img = skimage.transform.resize(image = every_image,output_shape=(image_size,image_size))
    
    if(img_count==1):
      x_train_every_image = img[:,:,:3]
    else:
      x_train_every_image = np.vstack((x_train_every_image,img[:,:,:3]))
    
  y = classes[every_class]*np.ones(shape=(len(every_collection.files),),dtype='int32')
  if(class_count==1):
    x_train = x_train_every_image
    y_train=y
  else:
    y_train=np.concatenate((y_train,y))
    x_train = np.vstack((x_train,x_train_every_image))
  
  
  tot_img_count +=img_count
  img_count=0

x_train=x_train.reshape(tot_img_count,image_size,image_size,3)

In [7]:
x_train.shape

(4750, 128, 128, 3)

In [8]:
class_count

12

In [0]:
y_train = to_categorical(y=y_train,num_classes=12)

In [0]:
x_training,x_validate,y_training,y_validate = train_test_split(x_train,y_train,test_size=0.2,shuffle=True)

# Set the CNN model 

In [11]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.layers import Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

model = Sequential()
model.add(convolutional.Conv2D(input_shape=(128,128,3),filters=20,kernel_size=4,strides=4,data_format='channels_last',activation='relu'))
model.add(convolutional.Conv2D(input_shape=(20,32,32,3),filters=20,kernel_size=3,strides=2,data_format='channels_last',activation='relu'))
model.add(convolutional.Conv2D(input_shape=(20,15,15,3),filters=20,kernel_size=3,strides=1,data_format='channels_last',activation='relu'))

model.add(pooling.AvgPool2D(pool_size=3,strides=2,data_format='channels_last',input_shape=(20,15,15,3)))
model.add(pooling.AvgPool2D(pool_size=3,strides=1,data_format='channels_last',input_shape=(20,7,7,3)))
model.add(Flatten())
model.add(Dense(units=3940,activation='relu',input_shape=()))
model.add(Dropout(rate=0.2))
model.add(BatchNormalization())
model.add(Dense(units=600,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(BatchNormalization())
model.add(Dense(units=50,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(BatchNormalization())
model.add(Dense(units=12,activation='softmax'))
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 20)        980       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 20)        3620      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 20)        3620      
_________________________________________________________________
average_pooling2d_1 (Average (None, 6, 6, 20)          0         
_________________________________________________________________
average_pooling2d_2 (Average (None, 4, 4, 20)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 320)               0         


In [12]:

opt = Adam(lr=0.001)

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(x_training, y_training, epochs = 90, validation_data = (x_validate,y_validate),batch_size = 32)

Train on 3800 samples, validate on 950 samples
Epoch 1/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.2080 - acc: 0.9266 - val_loss: 0.4303 - val_acc: 0.8821
Epoch 2/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.2064 - acc: 0.9268 - val_loss: 0.4219 - val_acc: 0.8758
Epoch 3/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.2061 - acc: 0.9250 - val_loss: 0.5768 - val_acc: 0.8537
Epoch 4/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.2018 - acc: 0.9284 - val_loss: 0.4298 - val_acc: 0.8642
Epoch 5/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.1931 - acc: 0.9308 - val_loss: 0.6186 - val_acc: 0.8505
Epoch 6/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.1889 - acc: 0.9271 - val_loss: 0.7506 - val_acc: 0.8326
Epoch 7/90
3800/3800 [==============================] - 8s 2ms/step - loss: 0.1785 - acc: 0.9345 - val_loss: 1.1124 - val_acc: 0.7537
Epoch 8/90
3800

In [0]:
model.save_weights('model_cnn.h5')

In [0]:
model.load_weights('model_cnn.h5')

In [19]:
loss, acc = model.evaluate(x_validate,  y_validate, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 88.11%
